In [199]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier

In [200]:
train_data = pd.read_csv("datasets/train.csv")
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [201]:
# train_data.describe()
# prepare data
# features = ["Age"]
features = ["Pclass", "Sex", "SibSp", "Parch", "Embarked"]
train_data.dropna(subset=features, inplace=True, axis="rows")
# train_data["Age"] = train_data["Age"].round().astype(int)
train_data.head()


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [202]:
# print(np.any(np.isposinf(train_data["Age"])))
# print(np.any(np.isneginf(train_data["Age"])))
# print(np.where(np.isnan(train_data["Age"])))
# train_data["Age"].describe()
# train_data["Age"].to_csv("ages.csv", index = False)

In [203]:
test_data = pd.read_csv("datasets/test.csv")

In [204]:
y = train_data["Survived"]
X = pd.get_dummies(train_data[features])
# X.describe()
X_test = pd.get_dummies(test_data[features])
rndForestClassifier = RandomForestClassifier(
    n_estimators=100, max_depth=5, random_state=1
)
rndForestClassifier.fit(X, y)
predictions = rndForestClassifier.predict(X_test)


In [205]:

submission_data = pd.DataFrame({"PassengerId": test_data.PassengerId, "Survived": predictions})
submission_data.to_csv("datasets/submission.csv", index=False)
